# Flicker Crawler

In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import random
import time
import re
import os
import base64

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
import wget

## Import breeds from txt file

In [ ]:
final_breeds = []

path = os.getcwd()
my_file = open(os.path.join(path ,"Breeds_File.txt"), "r")
content = my_file.read()
content_list = content.split("\n")
my_file.close()

for breed in content_list:
        word = breed.replace("_", " ").replace("-", " ").replace(" ", "")
        word= word[:1].upper() + word[1:]
        final_breeds.append(word)
        
print(final_breeds)

## Selenium driver setup

In [ ]:
chrome_options = webdriver.ChromeOptions()
s = Service('C:/Users/Asus/Documents/WebDriver/chromedriver.exe')
driver = webdriver.Chrome(service=s)
driver.get('https://www.flickr.com/')
time.sleep(5)

In [ ]:
search_URL = "https://www.flickr.com/search/?text="
main_URL = "https://www.flickr.com/"
download_URl = "sizes/l/"
show_all_extension = "&view_all=1"

## Scroll Down Automation

In [ ]:
def scroll_down(driver):
    img_urls = []
    try:
        # Get scroll height.
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            time.sleep(random.randint(1,3))
            page_source = driver.page_source
            soup = bs(page_source, 'html.parser')
            links = soup.find_all('a',{'class':'overlay'}, href=True)
            for link in links:
                img_urls.append("https://www.flickr.com"+link['href'])

            time.sleep(random.randint(2,4))

            #Remove Duplicates Of Img URLs
            img_urls = list( dict.fromkeys(img_urls))
            print("Images: %s" % (len(img_urls)))
            if(len(img_urls)>15000):
                break
            # Scroll down to the bottom.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load the page.
            time.sleep(random.randint(3,6))
            # Calculate new scroll height and compare with last scroll height.
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                time.sleep(random.randint(7,12)) 
                try:
                    load_more_clickable = WebDriverWait(driver,60).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Load more results')]"))).click()
                    print("Clicked Load More...")
                except:
                    print("End Of Page..")
                    break
            last_height = new_height
            
    except ElementNotInteractableException:
        raise Exception("Exception -> end of page")
    
    return img_urls

## Flicker Crawling Functions

In [ ]:
def flicker_image_crawel(url):
    driver.get(url)
    time.sleep(random.randint(3,6))
    page_source = driver.page_source
    soup = bs(page_source, 'html.parser')
    download_nav = soup.find('div',{'class':'view photo-engagement-view'})
    download_icon = download_nav.find('div',{'class':'engagement-item download'})
    download_link = download_icon.find('a', href=True)
    img_url = flicker_image_downloader(main_URL+download_link['href'])
    return img_url

def flicker_image_downloader(url):
    driver.get(url)
    time.sleep(random.randint(3,6))
    page_source = driver.page_source
    soup = bs(page_source, 'html.parser')
    try:
        img_div = soup.find('div',{'id':'allsizes-photo'})
        img = img_div.find('img')['src']
    except:
        pass
    return img

def flicker_image_download_link(url):
    driver.get(url+download_URl)
    time.sleep(random.randint(2,4))
    page_source = driver.page_source
    soup = bs(page_source, 'html.parser')
    try:
        img_div = soup.find('div',{'id':'allsizes-photo'})
        img = img_div.find('img')['src']
    except:
        print("Cant get image link!")
        pass
    return img

In [ ]:
def download_image(url, folder_name, num):
    # write image to file
    reponse = requests.get(url)
    if reponse.status_code==200:
        path = os.getcwd()
        path = os.path.join(path, folder_name)
        with open(os.path.join(path, folder_name + str(num)+".jpg"), 'wb') as file:
            file.write(reponse.content)
            
def create_folder(query):
    if not os.path.exists(os.getcwd() + '/' + query):
        path = os.getcwd()
        path = os.path.join(path, query)
        os.mkdir(path)
        print(query + " Folder Created")
    else:
        print(query + " Folder Already Exist")
        
def save_images(images, query):
    counter = 0
    path = os.getcwd()
    path = os.path.join(path, query)
    for img in images:
        if(img):
            save_as = os.path.join(path ,query + str(counter) +'.jpg')
            wget.download(img, save_as)
            counter += 1
        else:
            print("Cant retrive image will continue...")
            pass
    
    print("%s Images Saved! | Total of: %s Images" %(query,counter))
    
def save_single_images(img, query):
    if(img):
        counter = 0
        path = os.getcwd()
        path = os.path.join(path, query)
        save_as = os.path.join(path ,query + str(counter) +'.jpg')
        print(img)
        wget.download(img, save_as)
    else:
        print("Cant retrive image will continu...")
    
    print("%s Images Saved!" %(query))

## Use The Automation For Each Breed

In [ ]:
wait = WebDriverWait(driver, 10)

for breed in final_breeds:
    
    print("Search images for: " + breed)
    img_urls = []
    img_download_links = []
    query = breed.replace(" ", "")
    page = driver.get(search_URL + query + show_all_extension) 

    img_urls = scroll_down(driver)
    print("Images: %s" % (len(img_urls)))
    print(img_urls)
    
    count = 0
    for img in img_urls:
        img_download_links.append(flicker_image_download_link(img))
    
    print("Len:%s"%(len(img_download_links)))
    
    #create path if dosent exsist
    create_folder(query)
    
    #save the images
    save_images(img_download_links,query)

### In this project all the data came from scrapping and the crawling is used for educational purpose only.